In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [2]:
from keras import Input # A tensor that is used to built a model just by knowing its inputs and outputs
from keras.layers import Dense, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error
from keras.callbacks import EarlyStopping

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn import metrics
from sklearn.model_selection import train_test_split


from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os   # This module is used for interacting with the operating system. It provides a way to work with files and directories.
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import tensorflow.keras.backend as K

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load data and setup directories ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

#'r' before the string indicates a raw string ('\' are treated as literal and not escape characters)
embeddings_path = r"embeddings"
result_path = r"results"

if not os.path.exists(embeddings_path):
    os.makedirs(embeddings_path)
if not os.path.exists(result_path):
    os.makedirs(result_path)

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
# --- Prepare input data ---
input_data_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv", sep="\t")   # read .tsv file into memory
input_data = input_data_frame.values  # retrieve values as numpy array
print ("Shape of input data: ", input_data.shape) # rows are genes and columns are samples
print(input_data)

input_data = np.transpose(input_data) # rows are samples and columns are genes
input_data = np.delete(input_data, (0), axis=0) # delete first row of the data matrix
print ("Shape of prepared input data: ", input_data.shape)
print(input_data)

input_data = np.asarray(input_data).astype("float32") # Every character/number encoded with 32 Bit
print("--- Converted to Float32 ---")
print(input_data)

#input_data = K.constant(input_data) # convert to Keras tensor to achieve

Shape of input data:  (17162, 1080)
[['A1BG' 0.1719444908272692 0.7300083449184781 ... 0.614468050540426
  1.3458846686508648 0.7186659277414379]
 ['A1CF' -0.814924951368837 2.386837959210663 ... 3.016626468680079
  -0.814924951368837 0.2356343444931301]
 ['A2M' -0.946474342215696 -0.5321448682087864 ... 0.5601782905367024
  1.512226893819039 -0.6446442624050204]
 ...
 ['DHCR7' 0.9104445032812696 0.3842357893348481 ... 0.7718469491250562
  -0.4217551872682497 -0.970509103357415]
 ['TMEM45B' -1.9635765960806173 1.268061022702539 ... -0.3477150909879061
  0.4043269887748967 -0.5444141861092432]
 ['GPR160' 0.6427457224400778 1.759067645972089 ... 0.3169998035848742
  -0.7956005603903231 -0.1338926768376846]]
Shape of prepared input data:  (1079, 17162)
[[0.1719444908272692 -0.814924951368837 -0.946474342215696 ...
  0.9104445032812696 -1.9635765960806173 0.6427457224400778]
 [0.7300083449184781 2.386837959210663 -0.5321448682087864 ...
  0.3842357893348481 1.268061022702539 1.759067645972

In [6]:
# --- Architecture Specifications ---
encoding1_dim = 2000
encoding2_dim = 200
encoding3_dim = 40
latent_dim = 10

In [7]:
def sampling(args):
  z_mean, z_log_var = args
  # retrieve tensor with as many random elements as the batch has elements
  # Each element is a vector with as many dimensions as the latent variable
  # K.shape(z_mean)[0] <=> gives the first element of the shape, which is batch_size
  # K.shape(z_mean)[1] <=> gives the second element of the shape, which is number of dimensions of the data objects
  epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], tf.shape(z_mean)[1]), mean=0., stddev=1.0)
  return z_mean + tf.exp(z_log_var / 2) * epsilon # epsilon is scaled by standard deviation and shifted by mean

In [8]:
for i in range(6, 11):
    # --- Create encoder ---
    # We create the model by chaining layer calls to specify the forward pass of the model
    # The final model is then created from inputs and outputs of the those calls!
    input_dim = input_data.shape[1]
    encoder_input = Input(shape=(input_dim,)) # A tensor object that embodies the network input
    #h = Dense(encoding1_dim, activation=keras.layers.LeakyReLU(alpha=0.0001))(encoder_inputs)
    h = Dense(encoding1_dim, activation='sigmoid')(encoder_input)
    h = Dense(encoding2_dim, activation='sigmoid')(h)
    h = Dense(encoding3_dim, activation='sigmoid')(h)
    # Layers to learn mean/variance of the multivariate normal distribution that governs Z
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)

    # Sampling that that encapsulates the sampling function
    z = Lambda(sampling)([z_mean, z_log_var])

    encoder = Model(encoder_input, [z_mean, z_log_var, z])
    encoder.summary()
    # -------------------------

    # --- Create Decoder ---
    latent_input = Input(shape=(latent_dim,))
    h_decoder = Dense(encoding3_dim, activation='sigmoid')(latent_input)
    h_decoder = Dense(encoding2_dim, activation='sigmoid')(h_decoder)
    h_decoder = Dense(encoding1_dim, activation='sigmoid')(h_decoder)
    decoder_output = Dense(input_dim)(h_decoder)

    decoder = Model(latent_input, decoder_output)
    decoder.summary()
    # -------------------------------

    # --- Create VAE ---
    # Samplings from the latent distribution learned by encoder serve as input to the decoder
    vae_output = decoder(encoder(encoder_input)[2])
    vae = Model(inputs=encoder_input, outputs=vae_output)
    vae.summary()

    # --- Specify custom loss function
    # 'axis' specifies the dimension along which the operation is applied
    # For axis=0, the summation is performed along the rows, so summing all values in a column for every column
    # For axis=-1, the operation is performed along the last axis, which is here the dimensions of the data object
    # By default, in the mean squared error loss the mean is applied to axis=-1, resulting in a tensor with 1 dimensions less
    reconstruction_loss = mean_squared_error(encoder_input, vae_output)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(reconstruction_loss + kl_loss) # Mean is applied to entire tensor by default, resulting in a scaler
    vae.add_loss(vae_loss)
    # ---------------------------------

    # initialize a new run (<=> single unit of computation)
    run = wandb.init(project="VAE_fitting",
                     # Set the project where this run will be logged
                     name=f"tryout",  # We pass a run name otherwise it’ll be randomly assigned
                     # Track hyperparameters and run metadata
                     config={
                         "optimizer": "adam",
                         "training_set": "BRCA",
                         "validation split": 0.1,
                         "patience": 25,
                         "min_delta": 0.01,
                         "test_set": "none",
                         "batch_size": 64,
                         "layer_dimensions": "2000, 200, 40, 10",
                         "activation_functions": "sigmoid, sigmoid, linear",
                         "model_z": "Gaussian distribution",
                         "reconstruction_loss": "mse"
                     })

    vae.compile(optimizer=Adam(learning_rate=0.000001))
    x, x_val = train_test_split(input_data, test_size=0.1, train_size=0.9)

    callback_metrics = WandbMetricsLogger()
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=25, min_delta=0.01, restore_best_weights=True)
    vae.fit(x, x, validation_data=(x_val, x_val), epochs=1000, verbose=1, batch_size=64, shuffle=True, callbacks=[callback_metrics, early_stopping])

    # Save embedded encodings
    # Take the mean values as predictions
    z_mean, z_log_var, _ = encoder.predict(input_data)
    # Sample embedding from the gaussian distribution for each element
    sampled_embeddings = tf.random.normal(shape=(tf.shape(z_mean)[0], tf.shape(z_mean)[1]), mean=z_mean, stddev=tf.exp(z_log_var / 2))
    print("Shape of means: ", z_mean.shape)
    print("Shape of sampled embeddings: ", sampled_embeddings.shape)
    if not os.path.exists("{p}/VAE_Z_EM_Mean_{i}.txt".format(p=result_path, i=i)):
        os.mknod("{p}/VAE_Z_EM_Mean_{i}.txt".format(p=result_path, i=i))
    if not os.path.exists("{p}/VAE_Z_EM_Sampled_{i}.txt".format(p=result_path, i=i)):
        os.mknod("{p}/VAE_Z_EM_Sampled_{i}.txt".format(p=result_path, i=i))
    np.savetxt("{p}/VAE_Z_EM_Mean_{i}.txt".format(p=result_path, i=i), z_mean)
    np.savetxt("{p}/VAE_Z_EM_Sampled_{i}.txt".format(p=result_path, i=i), sampled_embeddings)


    # Mark the run as finished
    wandb.finish()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17162)]              0         []                            
                                                                                                  
 dense (Dense)               (None, 2000)                 3432600   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 dense_1 (Dense)             (None, 200)                  400200    ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 40)                   8040      ['dense_1[0][0]']         

wandb: Currently logged in as: cosybio-compsysmed. Use `wandb login --relogin` to force relogin


Epoch 1/1000
16/16 [==============================] - 6s 56ms/step - loss: 4.0886 - val_loss: 4.0739
Epoch 2/1000
16/16 [==============================] - 0s 22ms/step - loss: 4.0221 - val_loss: 4.0222
Epoch 3/1000
16/16 [==============================] - 0s 22ms/step - loss: 3.9823 - val_loss: 3.9719
Epoch 4/1000
16/16 [==============================] - 0s 21ms/step - loss: 3.9124 - val_loss: 3.9217
Epoch 5/1000
16/16 [==============================] - 0s 21ms/step - loss: 3.8770 - val_loss: 3.8720
Epoch 6/1000
16/16 [==============================] - 0s 22ms/step - loss: 3.8195 - val_loss: 3.8243
Epoch 7/1000
16/16 [==============================] - 0s 22ms/step - loss: 3.7639 - val_loss: 3.7766
Epoch 8/1000
16/16 [==============================] - 0s 21ms/step - loss: 3.6985 - val_loss: 3.7301
Epoch 9/1000
16/16 [==============================] - 0s 21ms/step - loss: 3.6714 - val_loss: 3.6849
Epoch 10/1000
16/16 [==============================] - 0s 21ms/step - loss: 3.6222 - val_lo

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▇▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,435
epoch/learning_rate,0.0
epoch/loss,1.02516
epoch/val_loss,1.04593


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 17162)]              0         []                            
                                                                                                  
 dense_9 (Dense)             (None, 2000)                 3432600   ['input_3[0][0]']             
                                                          0                                       
                                                                                                  
 dense_10 (Dense)            (None, 200)                  400200    ['dense_9[0][0]']             
                                                                                                  
 dense_11 (Dense)            (None, 40)                   8040      ['dense_10[0][0]']      

...

Epoch 1/1000
16/16 [==============================] - 4s 47ms/step - loss: 5.4536 - val_loss: 5.4590
Epoch 2/1000
16/16 [==============================] - 0s 23ms/step - loss: 5.3651 - val_loss: 5.3755
Epoch 3/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.2832 - val_loss: 5.2939
Epoch 4/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.2180 - val_loss: 5.2160
Epoch 5/1000
16/16 [==============================] - 0s 22ms/step - loss: 5.1163 - val_loss: 5.1407
Epoch 6/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.0371 - val_loss: 5.0645
Epoch 7/1000
16/16 [==============================] - 0s 21ms/step - loss: 4.9500 - val_loss: 4.9898
Epoch 8/1000
16/16 [==============================] - 0s 21ms/step - loss: 4.8632 - val_loss: 4.9144
Epoch 9/1000
16/16 [==============================] - 0s 21ms/step - loss: 4.7911 - val_loss: 4.8433
Epoch 10/1000
16/16 [==============================] - 0s 22ms/step - loss: 4.7278 - val_lo

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,428
epoch/learning_rate,0.0
epoch/loss,1.01441
epoch/val_loss,1.06294


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 17162)]              0         []                            
                                                                                                  
 dense_36 (Dense)            (None, 2000)                 3432600   ['input_9[0][0]']             
                                                          0                                       
                                                                                                  
 dense_37 (Dense)            (None, 200)                  400200    ['dense_36[0][0]']            
                                                                                                  
 dense_38 (Dense)            (None, 40)                   8040      ['dense_37[0][0]']     

Epoch 1/1000
16/16 [==============================] - 3s 46ms/step - loss: 5.8978 - val_loss: 5.8825
Epoch 2/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.8432 - val_loss: 5.8141
Epoch 3/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.7725 - val_loss: 5.7497
Epoch 4/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.6888 - val_loss: 5.6847
Epoch 5/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.6411 - val_loss: 5.6197
Epoch 6/1000
16/16 [==============================] - 0s 22ms/step - loss: 5.5525 - val_loss: 5.5534
Epoch 7/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.5030 - val_loss: 5.4869
Epoch 8/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.4306 - val_loss: 5.4219
Epoch 9/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.3522 - val_loss: 5.3564
Epoch 10/1000
16/16 [==============================] - 0s 21ms/step - loss: 5.3008 - val_lo

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,656
epoch/learning_rate,0.0
epoch/loss,1.04911
epoch/val_loss,1.07923
